<a href="https://colab.research.google.com/github/shimo23333/generative_ai/blob/main/%E4%BD%BF%E7%94%A8_CLIP_%E6%A8%A1%E5%9E%8B%E5%8F%AF%E8%A6%96%E5%8C%96%E4%B8%AD%E8%8B%B1%E6%96%87%E6%83%85%E7%B7%92%E8%A9%9E%E5%BD%99%E7%9A%84%E8%AA%9E%E6%84%8F%E5%B7%AE%E7%95%B02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### CLIP 多語言情緒詞 → 圖像 → 色票分析

# 1.安裝必要套件
先安裝會用到的四個核心工具，CLIP 負責語意處理、colorthief 負責從圖像擷取主色、googletrans 用來翻譯情緒詞，OpenCV 把 RGB 轉成 Lab 色彩空間。

In [9]:
# 區塊 1：安裝必要套件
!pip install git+https://github.com/openai/CLIP.git # 安裝 OpenAI 的 CLIP 模型
!pip install colorthief # 安裝色票擷取套件
!pip install googletrans==4.0.0-rc1 # 安裝 Google 翻譯 API 套件
!pip install opencv-python # 安裝 OpenCV，進行色彩轉換

  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-zlxmx6lu
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-zlxmx6lu
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done


# 2.引入套件
把所有需要用到的 Python 套件載入，並且為了圖表美觀，設定可以顯示中文字的字體。

In [10]:
# 區塊 2：引入套件
import clip
import torch
from PIL import Image
import requests
import os
from io import BytesIO
from colorthief import ColorThief
from googletrans import Translator
import matplotlib.pyplot as plt
import numpy as np
import cv2
import seaborn as sns

# 設定中文字體（避免亂碼）
plt.rcParams['font.family'] = 'Noto Sans CJK JP'
plt.rcParams['axes.unicode_minus'] = False

# 3.初始化 CLIP 與翻譯工具
這裡載入 CLIP 模型，選用 ViT-B/32 版本。如果使用者有 GPU 就用 CUDA，沒有就用 CPU。翻譯器 Translator 用來翻多語情緒詞。

In [11]:
# 區塊 3：設定模型與語言
translator = Translator()
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)


# 4：翻譯原始情緒詞（中文 → 英、日、韓）
定義一組情緒詞，並透過 Google 翻譯 API，自動翻成英文、日文、韓文。這樣每個中文情緒詞就會對應四種語言版本。

In [12]:
# 區塊 4：輸入原始中文詞 → 翻譯為中英日韓
emotion_words_zh = ["溫暖", "空虛", "混亂", "壓抑", "從容", "曖昧"]
def translate_words(words):
    results = []
    for word in words:
        en = translator.translate(word, src='zh-CN', dest='en').text
        ja = translator.translate(word, src='zh-CN', dest='ja').text
        ko = translator.translate(word, src='zh-CN', dest='ko').text
        results.append({"zh": word, "en": en, "ja": ja, "ko": ko})
    return results

translated_emotions = translate_words(emotion_words_zh)

# 5：用 CLIP 提取語意圖像
理想上這裡可以搭配 DALL·E 做語意生成，但這個版本使用 Unsplash 隨機抓對應詞的圖片來模擬圖像回傳（像是搜尋「warm」就可能抓到夕陽圖片）。

In [13]:
# 區塊 5：從 CLIP 擷取語意圖像
from torchvision.transforms import Compose, Resize, CenterCrop, ToTensor, Normalize

def retrieve_image(text, lang):
    prompt = f"{text}"
    with torch.no_grad():
        text_token = clip.tokenize([prompt]).to(device)
        text_embed = model.encode_text(text_token)
    # 模擬取得圖像（如有 DALL-E 可替換這裡）
    url = f"https://source.unsplash.com/400x400/?{text}"
    response = requests.get(url)
    image = Image.open(BytesIO(response.content))
    return image

# 6：從圖像中抓出主色（RGB）
用 ColorThief 擷取圖片的主色（即色票中最顯眼的一個 RGB 值），這是我們後面做色彩分析的關鍵。

In [14]:
# 區塊 6：色票分析 (使用 colorthief)
def get_main_color(image):
    buffer = BytesIO()
    image.save(buffer, format="PNG")
    buffer.seek(0)
    color_thief = ColorThief(buffer)
    dominant = color_thief.get_color(quality=1)
    return dominant


# 7：RGB → Lab，並畫出顏色
rgb_to_lab()：把 RGB 轉成 Lab 色彩空間，利於做數值比較。

plot_colors()：將每種語言的主色畫出色票對照圖，一眼看出色彩差異。

In [15]:
# 區塊 7：色彩轉 Lab (並視覺化)
def rgb_to_lab(color):
    rgb = np.uint8([[color]])
    lab = cv2.cvtColor(rgb, cv2.COLOR_RGB2LAB)
    return lab[0][0]

def plot_colors(title, color_dict):
    fig, ax = plt.subplots(1, len(color_dict), figsize=(10, 3))
    for i, (label, rgb) in enumerate(color_dict.items()):
        ax[i].imshow([[rgb]], aspect='auto')
        ax[i].set_title(label)
        ax[i].axis('off')
    plt.suptitle(title)
    plt.show()

# 8：主流程（翻譯 → 抓圖 → 擷取色彩 → 顯示）
這是主要執行區塊，對每個情緒詞（如「溫暖」）跑一次完整流程：

翻譯多語詞彙

擷取對應圖片

擷取主色

畫出四個語言版本的色彩對照圖

In [16]:
# 區塊 8：主執行邏輯：逐一詞語 → 多語翻譯 → 生成圖像 → 色票 → 色彩對照圖
for item in translated_emotions:
    color_map = {}
    for lang in ["zh", "en", "ja", "ko"]:
        word = item[lang]
        img = retrieve_image(word, lang)
        rgb = get_main_color(img)
        lab = rgb_to_lab(rgb)
        color_map[f"{lang}: {word}"] = rgb
    plot_colors(f"『{item['zh']}』的多語色票對照", color_map)


UnidentifiedImageError: cannot identify image file <_io.BytesIO object at 0x7a74127bfbf0>